In [ ]:
import numpy as np
import networkx as nx
import cvxpy as cp

In [ ]:
def map_lp(J, h):
    # prepare data
    nodes = list(range(len(J)))
    G = nx.from_numpy_matrix(J)
    n = len(J)
    edges = list(G.edges())
    
    x_beliefs = cp.Variable((len(nodes), 2), nonneg=True)
    edge_beliefs = []
    for edge in edges:
        edge_beliefs.append(cp.Variable((2, 2), nonneg=True))

    objective = 0
    constraints = []

    # add field
    for node in nodes:
        field = h[node]
        if field <= 0:
            objective -= field * x_beliefs[node, 1]
            objective += field * x_beliefs[node, 0]
        else:
            objective += field * x_beliefs[node, 1]
            objective -= field * x_beliefs[node, 0]

    # add pairwise interactions
    for edge in edges:
        i = edges.index(edge)
        a = edge[0]
        b = edge[1]
        objective -= J[a, b] * edge_beliefs[i][1, 1]
        objective -= J[a, b] * edge_beliefs[i][0, 0]
        objective += J[a, b] * edge_beliefs[i][1, 0]
        objective += J[a, b] * edge_beliefs[i][0, 1]

    # normalization constraints
    for node in nodes:
        constraints += [x_beliefs[node, 0] + x_beliefs[node, 1] == 1]
    
    # x marginalization constraints
    for edge in edges:
        i = edges.index(edge)
        a = edge[0]
        b = edge[1]
        marginal_left_0 = edge_beliefs[i][0, 0] + edge_beliefs[i][0, 1]
        constraints += [marginal_left_0 == x_beliefs[a, 0]]

        marginal_left_1 = edge_beliefs[i][1, 0] + edge_beliefs[i][1, 1]
        constraints += [marginal_left_1 == x_beliefs[a, 1]]

        marginal_right_0 = edge_beliefs[i][0, 0] + edge_beliefs[i][1, 0]
        constraints += [marginal_right_0 == x_beliefs[b, 0]]

        marginal_right_1 = edge_beliefs[i][0, 1] + edge_beliefs[i][1, 1]
        constraints += [marginal_right_1 == x_beliefs[b, 1]]

    
    prob = cp.Problem(cp.Minimize(objective), constraints)
    prob.solve()  # solver=cp.SCS,eps=1e-8

    return np.round(x_beliefs.value, 3)